In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt

from slicer_generation import *
from surface_solns import *

Initialize same params as in test.c

In [10]:
p = ImageSlicerParams(
    custom = 0,
    cylinder = 0,
    n_each = 1,
    n_rows = 1,
    n_cols = 1,
    angle_mode = 0,
    dalpha = 0,
    dbeta = 0,
    dgamma = 0,
    gamma_offset = 0,
    alpha_cen = 0,
    beta_cen = 0,
    gamma_cen = 0,
    dx = 10,
    dy = 10,
    c = -0.01,
    k = -0.9999,
    gx_width = 0,
    gx_depth = 0,
    gy_width = 0,
    gy_depth = 0
)

if p.c == 0:
    sag_func = tilted_plane_sag;
    critical_xy_func = tilted_plane_critical_xy;
    transfer_dist_func = tilted_plane_transfer;
    surf_normal_func = tilted_plane_surface_normal;
else:
    sag_func = conic_3d_sag;
    critical_xy_func = conic_3d_critical_xy;
    transfer_dist_func = conic_3d_transfer;
    surf_normal_func = conic_3d_surface_normal;

Slicer transfer

In [3]:
data_file = "../src/test_output.txt"
testxt, testyt, testl, testm, testn, testoutt, testoutln, testoutmn, testoutnn = np.loadtxt(data_file).T
"""
testxt = [testxt]
testyt = [testyt]
testl = [testl]
testm = [testm]
testn = [testn]
testoutt = [testoutt]
testoutln = [testoutln]
testoutmn = [testoutmn]
testoutnn = [testoutnn]
"""

'\ntestxt = [testxt]\ntestyt = [testyt]\ntestl = [testl]\ntestm = [testm]\ntestn = [testn]\ntestoutt = [testoutt]\ntestoutln = [testoutln]\ntestoutmn = [testoutmn]\ntestoutnn = [testoutnn]\n'

In [4]:
testxt[0]

-3.0

In [7]:
pyout[0,0]

-0.08229941905597576

In [8]:
npts = len(testoutt)
pyout = np.empty((4, npts))

zmin, zmax = find_global_extrema_slicer(p, sag_func, critical_xy_func)

for i in range(npts):
    ray_in = RayIn(testxt[i], testyt[i], testl[i], testm[i], testn[i])
    ray_out = ray_trace_slicer(ray_in, zmin, zmax, p, sag_func, transfer_dist_func, surf_normal_func)
    pyout[0,i] = ray_out.t
    pyout[1,i] = ray_out.ln
    pyout[2,i] = ray_out.mn
    pyout[3,i] = ray_out.nn
    print("    >>>going to next")

bounds: (-1.0, -9.0, 0.0, 4.0)
checking (0.0, 4.0)
t is nan
result is nan
    >>>going to next
bounds: (0.0, 4.0, 0.0, 3.0)
checking (0.0, 4.0)
t is -2.605690880562595
result is 2.5490720645393594e-13
    >>>going to next
bounds: (-1.0, 16.0, 0.0, 2.0)
checking (0.0, 2.0)
t is 74.99577099496595
result is nan
    >>>going to next
bounds: (0.0, 2.0, 0.0, 3.0)
checking (0.0, 2.0)
t is -2.913698237157094
result is 0.0753497951273987
checking (0.0, 3.0)
t is -2.835462552467883
result is 3.019806626980426e-13
    >>>going to next
bounds: (1.0, -9.0, 0.0, 4.0)
checking (1.0, 0.0)
t is 58.13322402224421
result is nan
checking (1.0, 1.0)
t is 53.26965701754402
result is nan
checking (1.0, 2.0)
t is 48.368923034068764
result is nan
checking (1.0, 3.0)
t is 43.37711243607286
result is nan
checking (1.0, 4.0)
t is 38.194270389692484
result is nan
checking (0.0, 4.0)
t is 43.40354045890245
result is nan
    >>>going to next
bounds: (0.0, 4.0, 0.0, 3.0)
checking (0.0, 4.0)
t is -2.665817246750398
re

In [10]:
for i in range(npts):
    if abs(testoutt[i] - pyout[0,i]) > 1e-8:
        print("Index {} is wrong".format(i))
    elif np.isnan(testoutt[i]) ^ np.isnan(pyout[0,i]):
        print("Index {} both values should be nan".format(i))

Transfer distance

In [3]:
data_file = "../src/test_output.txt"
testxt, testyt, testl, testm, testn, testoutt = np.loadtxt(data_file).T

In [11]:
npts = len(testoutt)
pyout = np.empty(npts)
for i in range(npts):
    pyout[i] = transfer_dist_func(testxt[i], testyt[i], testl[i], testm[i], testn[i], p.c, p.k, p.alpha_cen, p.beta_cen, p.gamma_cen)

In [12]:
for i in range(npts):
    if abs(testoutt[i] - pyout[i]) > 1e-8:
        print("Index {} is wrong".format(i))
    elif np.isnan(testoutt[i]) ^ np.isnan(pyout[i]):
        print("Index {} both values should be nan".format(i))

Index 0 is wrong
Index 1 is wrong
Index 2 is wrong
Index 3 is wrong
Index 4 is wrong
Index 5 is wrong
Index 6 is wrong
Index 7 is wrong
Index 8 is wrong
Index 9 is wrong
Index 10 is wrong
Index 11 is wrong
Index 12 is wrong
Index 13 is wrong
Index 14 is wrong
Index 15 is wrong
Index 16 is wrong
Index 17 is wrong
Index 18 is wrong
Index 19 is wrong
Index 20 is wrong
Index 21 is wrong
Index 22 is wrong
Index 23 is wrong
Index 24 is wrong
Index 25 is wrong
Index 26 is wrong
Index 27 is wrong
Index 28 is wrong
Index 29 is wrong
Index 30 is wrong
Index 31 is wrong
Index 32 is wrong
Index 33 is wrong
Index 34 is wrong
Index 35 is wrong
Index 36 is wrong
Index 37 is wrong
Index 38 is wrong
Index 39 is wrong
Index 40 is wrong
Index 41 is wrong
Index 42 is wrong
Index 43 is wrong
Index 44 is wrong
Index 45 is wrong
Index 46 is wrong
Index 47 is wrong
Index 48 is wrong
Index 49 is wrong
Index 50 is wrong
Index 51 is wrong
Index 52 is wrong
Index 53 is wrong
Index 54 is wrong
Index 55 is wrong
In

Slicer sag

In [11]:
data_file = "../src/test_output.txt"
testx, testy, testout = np.loadtxt(data_file).T

In [12]:
npts = len(testout)
pyout = np.empty(npts)
for i in range(npts):
    pyout[i] = make_image_slicer(testx[i], testy[i], p, [], sag_func)

Validate outputs

In [13]:
for i in range(npts):
    if abs(testout[i] - pyout[i]) > 1e-8:
        print("Index {} is wrong".format(i))
    elif np.isnan(testout[i]) ^ np.isnan(pyout[i]):
        print("Index {} both values should be nan".format(i))